In [47]:
import scipy
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import SVHN
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
#from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import torchvision.models as models
import torch.optim as optim
from torch.optim import lr_scheduler
import copy
#%run Preprocessing_SVHN_data.ipynb
%matplotlib inline
from fastai.vision.widgets import *
import time
import pandas as pd

In [25]:
#! pip install mlcrate

In [26]:
from fastai.vision.all import *
from fastai.metrics import error_rate
import mlcrate as mlc
import matplotlib.pyplot as plt
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
#print(torch.cuda.current_device())

In [5]:
!nvidia-smi

Wed Nov  4 08:29:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    32W / 250W |     10MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
device

device(type='cuda', index=0)

In [8]:
transforms_final = transforms.Compose([ transforms.Resize(32),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.RandomVerticalFlip(),
                                        transforms.RandomRotation(30),
                                        #transforms.Grayscale(),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.4516),(0.1993))])

In [9]:
train_data = torchvision.datasets.SVHN("/home/jupyter/pytorch_datasets/train", split = "train", download = False, transform = transforms_final, target_transform = None)
validation_data = torchvision.datasets.SVHN("/home/jupyter/pytorch_datasets/test", split = "test", download = False, transform = transforms_final, target_transform = None)
extra_data = torchvision.datasets.SVHN("/home/jupyter/pytorch_datasets/extra", split = "extra", download = False, transform = transforms_final, target_transform = None)

In [47]:
len(train_data)

73257

In [48]:
train_labels = []
for x in range(0,len(train_data)):
    train_labels.append(train_data[x][1])

In [49]:
list(set(train_labels))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [50]:
validation_labels = []
for x in range(0,len(validation_data)):
    validation_labels.append(validation_data[x][1])

In [51]:
list(set(validation_labels))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [52]:
#train_set = torch.utils.data.ConcatDataset([train_set, extra_set])

In [53]:
batch_size = 64

In [54]:
train_loader = DataLoader(train_data, batch_size, shuffle=True, pin_memory = True, num_workers = 4)
val_loader = DataLoader(validation_data, batch_size, pin_memory = True, num_workers = 4)

In [55]:
#wide_resnet50_2 = models.wide_resnet50_2(pretrained = True)

In [56]:
#wide_resnet50_2

In [57]:
# time = mlc.time.Timer()
# learn = Learner.fit_one_cycle(train_loader,models.wide_resnet50_2(pretrained = True), metrics = [error_rate, accuracy])
# print("Train Cycle 1")
# learn.fit_one_cycle(8)
# learn.recorder.plot_losses() 
# print("Time taken: {}".format(time.since(0)))

In [58]:
image_datasets = {x: torchvision.datasets.SVHN("/home/jupyter/pytorch_datasets/" + x, split = x, download = False, transform = transforms_final, target_transform = None)
                  for x in ['train', 'test']}

In [59]:
image_datasets

{'train': Dataset SVHN
     Number of datapoints: 73257
     Root location: /home/jupyter/pytorch_datasets/train
     Split: train
     StandardTransform
 Transform: Compose(
                Resize(size=32, interpolation=PIL.Image.BILINEAR)
                RandomHorizontalFlip(p=0.5)
                RandomVerticalFlip(p=0.5)
                RandomRotation(degrees=[-30.0, 30.0], resample=False, expand=False)
                ToTensor()
                Normalize(mean=0.4516, std=0.1993)
            ),
 'test': Dataset SVHN
     Number of datapoints: 26032
     Root location: /home/jupyter/pytorch_datasets/test
     Split: test
     StandardTransform
 Transform: Compose(
                Resize(size=32, interpolation=PIL.Image.BILINEAR)
                RandomHorizontalFlip(p=0.5)
                RandomVerticalFlip(p=0.5)
                RandomRotation(degrees=[-30.0, 30.0], resample=False, expand=False)
                ToTensor()
                Normalize(mean=0.4516, std=0.1993)
          

In [60]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}

In [61]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f7f20020910>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f7f20020c90>}

In [62]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
dataset_sizes

{'train': 73257, 'test': 26032}

In [63]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [65]:
model_ft = models.wide_resnet50_2(pretrained = True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 10.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [66]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [69]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

Epoch 0/9
----------
train Loss: 1.2955 Acc: 0.5376
test Loss: 0.7987 Acc: 0.7143

Epoch 1/9
----------
train Loss: 0.7430 Acc: 0.7340
test Loss: 0.6365 Acc: 0.7711

Epoch 2/9
----------
train Loss: 0.6252 Acc: 0.7754
test Loss: 0.5743 Acc: 0.7901

Epoch 3/9
----------
train Loss: 0.5615 Acc: 0.7981
test Loss: 0.5393 Acc: 0.8059

Epoch 4/9
----------
train Loss: 0.5172 Acc: 0.8133
test Loss: 0.5099 Acc: 0.8162

Epoch 5/9
----------
train Loss: 0.4873 Acc: 0.8238
test Loss: 0.4803 Acc: 0.8274

Epoch 6/9
----------
train Loss: 0.4610 Acc: 0.8331
test Loss: 0.4728 Acc: 0.8317

Epoch 7/9
----------
train Loss: 0.4139 Acc: 0.8487
test Loss: 0.4469 Acc: 0.8390

Epoch 8/9
----------
train Loss: 0.4006 Acc: 0.8532
test Loss: 0.4328 Acc: 0.8440

Epoch 9/9
----------
train Loss: 0.3906 Acc: 0.8574
test Loss: 0.4286 Acc: 0.8456

Training complete in 12m 38s
Best val Acc: 0.845613


In [73]:
torch.save(model_ft, "/home/jupyter/pytorch_datasets/model_data")

In [27]:
model_ft_loaded = torch.load("/home/jupyter/pytorch_datasets/model_data")
model_ft_loaded.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [28]:
loader = transforms.Compose([ transforms.Resize(32),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.RandomVerticalFlip(),
                                        transforms.RandomRotation(30),
                                        #transforms.Grayscale(),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.4516),(0.1993))])

def image_loader(loader, image_name):
    image = Image.open(image_name)
    image = loader(image).float()
    image = image.clone().detach().requires_grad_(True)
    image = image.unsqueeze(0)
    image = image.cuda()
    #print(len(image))
    return image

In [31]:
pred = model_ft_loaded(image_loader(loader,"./picquora_test_cases/test28.png"))

In [38]:
pred

tensor([[-0.3707,  1.0375,  0.6579,  0.0876,  0.1342, -0.1483, -0.2453, -0.5878,
         -0.4530, -0.3244]], device='cuda:0', grad_fn=<AddmmBackward>)

In [37]:
torch.argmax(pred)

tensor(1, device='cuda:0')

In [39]:
predictions_dict = {}
image_name = []
predictions = []
for name in glob.glob(r'./picquora_test_cases/*.png'): 
    image_name.append(name)
    predictions.append(torch.argmax(model_ft_loaded(image_loader(loader,name))))
    

In [45]:
predictions_dict["Image_Name"] = image_name
predictions_dict["Prediction"] = predictions

In [46]:
predictions_dict

{'Image_Name': ['./picquora_test_cases/test38.png',
  './picquora_test_cases/test87.png',
  './picquora_test_cases/test81.png',
  './picquora_test_cases/test2.png',
  './picquora_test_cases/test42.png',
  './picquora_test_cases/test50.png',
  './picquora_test_cases/test28.png',
  './picquora_test_cases/test68.png',
  './picquora_test_cases/test58.png',
  './picquora_test_cases/test14.png',
  './picquora_test_cases/test24.png',
  './picquora_test_cases/test37.png',
  './picquora_test_cases/test47.png',
  './picquora_test_cases/test77.png',
  './picquora_test_cases/test21.png',
  './picquora_test_cases/test44.png',
  './picquora_test_cases/test52.png',
  './picquora_test_cases/test46.png',
  './picquora_test_cases/test35.png',
  './picquora_test_cases/test39.png',
  './picquora_test_cases/test31.png',
  './picquora_test_cases/test78.png',
  './picquora_test_cases/test41.png',
  './picquora_test_cases/test59.png',
  './picquora_test_cases/test4.png',
  './picquora_test_cases/test76.png',


In [48]:
df = pd.DataFrame(predictions_dict)

In [49]:
df

,Image_Name,Prediction
0,./picquora_test_cases/test38.png,"tensor(1, device='cuda:0')"
1,./picquora_test_cases/test87.png,"tensor(1, device='cuda:0')"
2,./picquora_test_cases/test81.png,"tensor(1, device='cuda:0')"
3,./picquora_test_cases/test2.png,"tensor(1, device='cuda:0')"
4,./picquora_test_cases/test42.png,"tensor(1, device='cuda:0')"
...,...,...
95,./picquora_test_cases/test74.png,"tensor(1, device='cuda:0')"
96,./picquora_test_cases/test65.png,"tensor(1, device='cuda:0')"
97,./picquora_test_cases/test73.png,"tensor(1, device='cuda:0')"
98,./picquora_test_cases/test97.png,"tensor(1, device='cuda:0')"
